Import Libraries

In [ ]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
%matplotlib inline

Dataset

In [ ]:
df = pd.read_csv('../input/heart-attack-analysis-prediction-dataset/heart.csv').drop_duplicates()
df

In [ ]:
import seaborn as sns
from matplotlib.pyplot import figure

figure(figsize = (10, 15))
sns.heatmap(df.isnull(), cbar=False)

In [ ]:
df.describe()

In [ ]:
df.info()

Histograms to evaluate features' correlation

In [ ]:
df.hist(figsize=(16,8))

Correlation matrix (heatmap)

In [ ]:
import seaborn as sns

plt.subplots(figsize=(10,10))
sns.heatmap(df.corr(), square=True, annot=True, fmt='.2f')
plt.show()

Features plots (overall)

In [ ]:
df.plot(subplots=True, figsize = (10, 20))
plt.show()

Models

In [ ]:
X = df.drop('output', axis = 1)
y = df['output']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
from sklearn.model_selection import GridSearchCV
knn_grid = {'n_neighbors': np.array(np.linspace(15, 35, 20), dtype='int')}
gs = GridSearchCV(knn, knn_grid, cv=5, n_jobs = -1)
gs.fit(X, y)
gs.best_params_, gs.best_score_

In [ ]:
def grid_plot(x, y, x_label, title, y_label='cross_val'):
    plt.figure(figsize=(12, 6))
    plt.grid(True)
    plt.plot(x, y, 'go-')
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)

In [ ]:
grid_plot(knn_grid['n_neighbors'], gs.cv_results_['mean_test_score'], 'n_neighbors', 'KNeighborsClassifier')

In [ ]:
knn = KNeighborsClassifier(n_neighbors=27, n_jobs=4)
knn.fit(X_train, y_train)

In [ ]:
y_knn = knn.predict(X_test)

In [ ]:
from sklearn import metrics
print(metrics.classification_report(y_test, y_knn))

Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
alg = LogisticRegression()

grid = {'penalty': ['l1', 'l2', 'elasticnet'],
        'C': np.array(np.logspace(-5, 5, num = 2), dtype='float'),
        }

gs = GridSearchCV(alg, grid, verbose=2, n_jobs = -1)
gs.fit(X_train, y_train)
gs.best_params_, gs.best_score_

In [ ]:
logreg = LogisticRegression(penalty='l2', C = 100000.0)

logreg.fit(X_train, y_train)

preds = logreg.predict(X_test)

print(metrics.classification_report(y_test, preds))

Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier(n_estimators=1000)
model_rf.fit(X_train, y_train)

# Сделаем предсказания
predictions_rf = model_rf.predict(X_test)
print(predictions_rf[:100])
predictions_rf.shape

In [ ]:
print(metrics.classification_report(y_test, predictions_rf))

Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)

In [ ]:
print(metrics.classification_report(y_test, preds))

Conclusion

Поскольку классы являются сбалансированными (что следует из графиков из раздела Features plots (overall), то для оценки предсказаний можем использовать метрику Accuracy. Наиболее высокий показатель - у модели "Наивный Байес".